In [42]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import gc

In [67]:
dat = pd.read_csv('dat_all.csv')
print dat.shape

(356255, 598)


In [68]:
for col in dat.drop('TARGET', axis=1).columns:
    if (col != 'SK_ID_CURR') & (dat[col].dtypes == 'object'):
        print col
        dat[col] = dat[col].astype('category').cat.codes

CODE_GENDER
EMERGENCYSTATE_MODE
FLAG_OWN_CAR
FLAG_OWN_REALTY
FONDKAPREMONT_MODE
HOUSETYPE_MODE
NAME_CONTRACT_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
NAME_INCOME_TYPE
NAME_TYPE_SUITE
OCCUPATION_TYPE
ORGANIZATION_TYPE
WALLSMATERIAL_MODE
WEEKDAY_APPR_PROCESS_START


In [69]:
train = dat[:307511]
test = dat[307511:]

train.drop('SK_ID_CURR', axis=1, inplace=True)
test_id = test['SK_ID_CURR']  # test_id
test.drop('SK_ID_CURR', axis=1, inplace=True)

del dat

train_X = train.drop('TARGET', axis=1)
train_y = train['TARGET']

test = test.drop('TARGET', axis=1)

del train
gc.collect()

/Users/jeremy/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/jeremy/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


21

In [7]:
print train_X.shape
print test.shape

(307511, 596)
(48744, 596)


In [8]:
train_y.value_counts()

0.0    282686
1.0     24825
Name: TARGET, dtype: int64

#### Try balance the training data set
1. over-sampling and under-sampling

In [21]:
sub_train = train_X[train_y==0].sample(n=150000, replace=False)
over_train = train_X[train_y==1].sample(n=100000, replace=True)

In [22]:
np.repeat([1, 0], [3, 2])

array([1, 1, 1, 0, 0])

In [23]:
train_X = sub_train.append(over_train)
train_y = np.repeat([0, 1], [150000, 100000])

print train_X.shape
print train_y.shape

(250000, 596)
(250000,)


#### Modeling

In [24]:
cat_feats = ['CODE_GENDER',
            'EMERGENCYSTATE_MODE',
            'FLAG_OWN_CAR',
            'FLAG_OWN_REALTY',
            'FONDKAPREMONT_MODE',
            'HOUSETYPE_MODE',
            'NAME_CONTRACT_TYPE',
            'NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS',
            'NAME_HOUSING_TYPE',
            'NAME_INCOME_TYPE',
            'NAME_TYPE_SUITE',
            'OCCUPATION_TYPE',
            'ORGANIZATION_TYPE',
            'WALLSMATERIAL_MODE',
            'WEEKDAY_APPR_PROCESS_START']

In [25]:
param = {# 'n_trees': 2000,
        'learning_rate': 0.02,
        'num_leaves': 34,
        'colsample_bytree': 0.9497036,
        'subsample': 0.8715623,
        'subsample_freq': 1,
        'max_depth': 8,
        'reg_alpha': 0.041545473,
        'reg_lambda': 0.0735294,
        'min_split_gain': 0.0222415,
        'min_child_weight': 60,
        'objective': 'binary',
        'metric': 'auc'}

In [26]:
from sklearn.model_selection import KFold

In [37]:
results = np.zeros((len(test), 5))
kf = KFold(n_splits=5, shuffle=True)

for n, (tr_idx, ts_idx) in enumerate(kf.split(train_X)):
    print 'Fold', n+1
    X_train, X_test = train_X.iloc[tr_idx], train_X.iloc[ts_idx]
    y_train, y_test = train_y[tr_idx], train_y[ts_idx]
    
    train_data = lgb.Dataset(X_train, label=y_train, feature_name='auto', 
                                  categorical_feature=cat_feats)

    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    model = lgb.train(param, train_data, 1600, valid_sets=[test_data], early_stopping_rounds=100)
    results[:, n] = model.predict(test)

Fold 1
[1]	valid_0's auc: 0.719681
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.726684
[3]	valid_0's auc: 0.72682
[4]	valid_0's auc: 0.72891
[5]	valid_0's auc: 0.729981
[6]	valid_0's auc: 0.731988
[7]	valid_0's auc: 0.732561
[8]	valid_0's auc: 0.732916
[9]	valid_0's auc: 0.733548
[10]	valid_0's auc: 0.733793
[11]	valid_0's auc: 0.73422
[12]	valid_0's auc: 0.73508
[13]	valid_0's auc: 0.737542
[14]	valid_0's auc: 0.737794
[15]	valid_0's auc: 0.738555
[16]	valid_0's auc: 0.739282
[17]	valid_0's auc: 0.740021
[18]	valid_0's auc: 0.740393
[19]	valid_0's auc: 0.740802
[20]	valid_0's auc: 0.741127
[21]	valid_0's auc: 0.741428
[22]	valid_0's auc: 0.741794
[23]	valid_0's auc: 0.742107
[24]	valid_0's auc: 0.742369
[25]	valid_0's auc: 0.742897
[26]	valid_0's auc: 0.743176
[27]	valid_0's auc: 0.743414
[28]	valid_0's auc: 0.743635
[29]	valid_0's auc: 0.744724
[30]	valid_0's auc: 0.745025
[31]	valid_0's auc: 0.745844
[32]	valid_0's auc: 0.746162
[33]	valid_0's 

[277]	valid_0's auc: 0.801748
[278]	valid_0's auc: 0.801853
[279]	valid_0's auc: 0.801974
[280]	valid_0's auc: 0.802127
[281]	valid_0's auc: 0.802216
[282]	valid_0's auc: 0.802318
[283]	valid_0's auc: 0.802424
[284]	valid_0's auc: 0.802542
[285]	valid_0's auc: 0.802645
[286]	valid_0's auc: 0.80278
[287]	valid_0's auc: 0.802893
[288]	valid_0's auc: 0.803006
[289]	valid_0's auc: 0.80311
[290]	valid_0's auc: 0.803235
[291]	valid_0's auc: 0.80334
[292]	valid_0's auc: 0.803446
[293]	valid_0's auc: 0.803534
[294]	valid_0's auc: 0.803655
[295]	valid_0's auc: 0.803776
[296]	valid_0's auc: 0.803895
[297]	valid_0's auc: 0.80402
[298]	valid_0's auc: 0.804129
[299]	valid_0's auc: 0.804219
[300]	valid_0's auc: 0.804329
[301]	valid_0's auc: 0.80443
[302]	valid_0's auc: 0.804527
[303]	valid_0's auc: 0.804629
[304]	valid_0's auc: 0.804731
[305]	valid_0's auc: 0.804831
[306]	valid_0's auc: 0.804919
[307]	valid_0's auc: 0.80503
[308]	valid_0's auc: 0.805156
[309]	valid_0's auc: 0.805276
[310]	valid_0's 

[552]	valid_0's auc: 0.825688
[553]	valid_0's auc: 0.825761
[554]	valid_0's auc: 0.825841
[555]	valid_0's auc: 0.825901
[556]	valid_0's auc: 0.825986
[557]	valid_0's auc: 0.82605
[558]	valid_0's auc: 0.826124
[559]	valid_0's auc: 0.826195
[560]	valid_0's auc: 0.826251
[561]	valid_0's auc: 0.826309
[562]	valid_0's auc: 0.826372
[563]	valid_0's auc: 0.826436
[564]	valid_0's auc: 0.826516
[565]	valid_0's auc: 0.826576
[566]	valid_0's auc: 0.826641
[567]	valid_0's auc: 0.826709
[568]	valid_0's auc: 0.826782
[569]	valid_0's auc: 0.826869
[570]	valid_0's auc: 0.826931
[571]	valid_0's auc: 0.827009
[572]	valid_0's auc: 0.827068
[573]	valid_0's auc: 0.827129
[574]	valid_0's auc: 0.827191
[575]	valid_0's auc: 0.827242
[576]	valid_0's auc: 0.827311
[577]	valid_0's auc: 0.827373
[578]	valid_0's auc: 0.827442
[579]	valid_0's auc: 0.827518
[580]	valid_0's auc: 0.827588
[581]	valid_0's auc: 0.827655
[582]	valid_0's auc: 0.827742
[583]	valid_0's auc: 0.827816
[584]	valid_0's auc: 0.827874
[585]	valid

[827]	valid_0's auc: 0.842077
[828]	valid_0's auc: 0.842121
[829]	valid_0's auc: 0.842194
[830]	valid_0's auc: 0.842255
[831]	valid_0's auc: 0.842299
[832]	valid_0's auc: 0.842349
[833]	valid_0's auc: 0.842402
[834]	valid_0's auc: 0.842453
[835]	valid_0's auc: 0.84251
[836]	valid_0's auc: 0.842546
[837]	valid_0's auc: 0.84259
[838]	valid_0's auc: 0.842655
[839]	valid_0's auc: 0.842712
[840]	valid_0's auc: 0.842776
[841]	valid_0's auc: 0.842831
[842]	valid_0's auc: 0.842901
[843]	valid_0's auc: 0.842951
[844]	valid_0's auc: 0.843007
[845]	valid_0's auc: 0.843083
[846]	valid_0's auc: 0.843142
[847]	valid_0's auc: 0.843176
[848]	valid_0's auc: 0.843229
[849]	valid_0's auc: 0.843279
[850]	valid_0's auc: 0.843298
[851]	valid_0's auc: 0.843357
[852]	valid_0's auc: 0.843402
[853]	valid_0's auc: 0.843458
[854]	valid_0's auc: 0.843507
[855]	valid_0's auc: 0.843564
[856]	valid_0's auc: 0.8436
[857]	valid_0's auc: 0.843644
[858]	valid_0's auc: 0.843691
[859]	valid_0's auc: 0.843754
[860]	valid_0'

[1098]	valid_0's auc: 0.855388
[1099]	valid_0's auc: 0.855441
[1100]	valid_0's auc: 0.855504
[1101]	valid_0's auc: 0.85556
[1102]	valid_0's auc: 0.855615
[1103]	valid_0's auc: 0.855662
[1104]	valid_0's auc: 0.855679
[1105]	valid_0's auc: 0.855721
[1106]	valid_0's auc: 0.855739
[1107]	valid_0's auc: 0.85579
[1108]	valid_0's auc: 0.855822
[1109]	valid_0's auc: 0.855855
[1110]	valid_0's auc: 0.85589
[1111]	valid_0's auc: 0.855933
[1112]	valid_0's auc: 0.855984
[1113]	valid_0's auc: 0.856012
[1114]	valid_0's auc: 0.856051
[1115]	valid_0's auc: 0.856113
[1116]	valid_0's auc: 0.856166
[1117]	valid_0's auc: 0.856204
[1118]	valid_0's auc: 0.856241
[1119]	valid_0's auc: 0.856275
[1120]	valid_0's auc: 0.856321
[1121]	valid_0's auc: 0.856352
[1122]	valid_0's auc: 0.856396
[1123]	valid_0's auc: 0.856446
[1124]	valid_0's auc: 0.856504
[1125]	valid_0's auc: 0.856546
[1126]	valid_0's auc: 0.8566
[1127]	valid_0's auc: 0.856637
[1128]	valid_0's auc: 0.856674
[1129]	valid_0's auc: 0.856726
[1130]	valid_

...

[394]	valid_0's auc: 0.817665
[395]	valid_0's auc: 0.817758
[396]	valid_0's auc: 0.817851
[397]	valid_0's auc: 0.817956
[398]	valid_0's auc: 0.818037
[399]	valid_0's auc: 0.81811
[400]	valid_0's auc: 0.818199
[401]	valid_0's auc: 0.818292
[402]	valid_0's auc: 0.818389
[403]	valid_0's auc: 0.818485
[404]	valid_0's auc: 0.818587
[405]	valid_0's auc: 0.818652
[406]	valid_0's auc: 0.818753
[407]	valid_0's auc: 0.81885
[408]	valid_0's auc: 0.818922
[409]	valid_0's auc: 0.818999
[410]	valid_0's auc: 0.819087
[411]	valid_0's auc: 0.819154
[412]	valid_0's auc: 0.819215
[413]	valid_0's auc: 0.8193
[414]	valid_0's auc: 0.819395
[415]	valid_0's auc: 0.819467
[416]	valid_0's auc: 0.819544
[417]	valid_0's auc: 0.81963
[418]	valid_0's auc: 0.819727
[419]	valid_0's auc: 0.819819
[420]	valid_0's auc: 0.819899
[421]	valid_0's auc: 0.81996
[422]	valid_0's auc: 0.820033
[423]	valid_0's auc: 0.820111
[424]	valid_0's auc: 0.820173
[425]	valid_0's auc: 0.82028
[426]	valid_0's auc: 0.820325
[427]	valid_0's a

[669]	valid_0's auc: 0.837231
[670]	valid_0's auc: 0.837275
[671]	valid_0's auc: 0.83732
[672]	valid_0's auc: 0.837381
[673]	valid_0's auc: 0.837452
[674]	valid_0's auc: 0.837522
[675]	valid_0's auc: 0.83757
[676]	valid_0's auc: 0.83762
[677]	valid_0's auc: 0.837673
[678]	valid_0's auc: 0.837732
[679]	valid_0's auc: 0.837785
[680]	valid_0's auc: 0.837833
[681]	valid_0's auc: 0.837886
[682]	valid_0's auc: 0.837956
[683]	valid_0's auc: 0.838013
[684]	valid_0's auc: 0.838065
[685]	valid_0's auc: 0.838114
[686]	valid_0's auc: 0.838187
[687]	valid_0's auc: 0.838244
[688]	valid_0's auc: 0.838295
[689]	valid_0's auc: 0.838369
[690]	valid_0's auc: 0.838444
[691]	valid_0's auc: 0.838505
[692]	valid_0's auc: 0.838559
[693]	valid_0's auc: 0.838623
[694]	valid_0's auc: 0.838692
[695]	valid_0's auc: 0.838757
[696]	valid_0's auc: 0.838816
[697]	valid_0's auc: 0.838873
[698]	valid_0's auc: 0.838914
[699]	valid_0's auc: 0.838979
[700]	valid_0's auc: 0.839029
[701]	valid_0's auc: 0.839079
[702]	valid_0

[944]	valid_0's auc: 0.851701
[945]	valid_0's auc: 0.851739
[946]	valid_0's auc: 0.851798
[947]	valid_0's auc: 0.851855
[948]	valid_0's auc: 0.851898
[949]	valid_0's auc: 0.851951
[950]	valid_0's auc: 0.851993
[951]	valid_0's auc: 0.852034
[952]	valid_0's auc: 0.85206
[953]	valid_0's auc: 0.852123
[954]	valid_0's auc: 0.852169
[955]	valid_0's auc: 0.852226
[956]	valid_0's auc: 0.852279
[957]	valid_0's auc: 0.852338
[958]	valid_0's auc: 0.852393
[959]	valid_0's auc: 0.852432
[960]	valid_0's auc: 0.852486
[961]	valid_0's auc: 0.85254
[962]	valid_0's auc: 0.852599
[963]	valid_0's auc: 0.852663
[964]	valid_0's auc: 0.852718
[965]	valid_0's auc: 0.852763
[966]	valid_0's auc: 0.85281
[967]	valid_0's auc: 0.852855
[968]	valid_0's auc: 0.85291
[969]	valid_0's auc: 0.852964
[970]	valid_0's auc: 0.85301
[971]	valid_0's auc: 0.853068
[972]	valid_0's auc: 0.85311
[973]	valid_0's auc: 0.853156
[974]	valid_0's auc: 0.853219
[975]	valid_0's auc: 0.853273
[976]	valid_0's auc: 0.853305
[977]	valid_0's 

[1212]	valid_0's auc: 0.863873
[1213]	valid_0's auc: 0.863897
[1214]	valid_0's auc: 0.863938
[1215]	valid_0's auc: 0.863997
[1216]	valid_0's auc: 0.864038
[1217]	valid_0's auc: 0.864092
[1218]	valid_0's auc: 0.864111
[1219]	valid_0's auc: 0.864155
[1220]	valid_0's auc: 0.864203
[1221]	valid_0's auc: 0.864245
[1222]	valid_0's auc: 0.864262
[1223]	valid_0's auc: 0.864284
[1224]	valid_0's auc: 0.864306
[1225]	valid_0's auc: 0.86435
[1226]	valid_0's auc: 0.864384
[1227]	valid_0's auc: 0.864433
[1228]	valid_0's auc: 0.864477
[1229]	valid_0's auc: 0.864526
[1230]	valid_0's auc: 0.864555
[1231]	valid_0's auc: 0.864591
[1232]	valid_0's auc: 0.864643
[1233]	valid_0's auc: 0.864692
[1234]	valid_0's auc: 0.864741
[1235]	valid_0's auc: 0.864792
[1236]	valid_0's auc: 0.864855
[1237]	valid_0's auc: 0.864896
[1238]	valid_0's auc: 0.864924
[1239]	valid_0's auc: 0.864981
[1240]	valid_0's auc: 0.865012
[1241]	valid_0's auc: 0.865049
[1242]	valid_0's auc: 0.865073
[1243]	valid_0's auc: 0.865116
[1244]	va

[1478]	valid_0's auc: 0.874268
[1479]	valid_0's auc: 0.874306
[1480]	valid_0's auc: 0.874355
[1481]	valid_0's auc: 0.874387
[1482]	valid_0's auc: 0.874397
[1483]	valid_0's auc: 0.874437
[1484]	valid_0's auc: 0.87447
[1485]	valid_0's auc: 0.874494
[1486]	valid_0's auc: 0.874536
[1487]	valid_0's auc: 0.874587
[1488]	valid_0's auc: 0.87464
[1489]	valid_0's auc: 0.874646
[1490]	valid_0's auc: 0.874674
[1491]	valid_0's auc: 0.874703
[1492]	valid_0's auc: 0.874733
[1493]	valid_0's auc: 0.874767
[1494]	valid_0's auc: 0.874812
[1495]	valid_0's auc: 0.874849
[1496]	valid_0's auc: 0.874889
[1497]	valid_0's auc: 0.874934
[1498]	valid_0's auc: 0.874977
[1499]	valid_0's auc: 0.875016
[1500]	valid_0's auc: 0.875056
[1501]	valid_0's auc: 0.875075
[1502]	valid_0's auc: 0.87511
[1503]	valid_0's auc: 0.875136
[1504]	valid_0's auc: 0.875163
[1505]	valid_0's auc: 0.875212
[1506]	valid_0's auc: 0.875218
[1507]	valid_0's auc: 0.875246
[1508]	valid_0's auc: 0.875287
[1509]	valid_0's auc: 0.875316
[1510]	vali

In [39]:
gc.collect()

77

In [40]:
result = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': np.mean(results, axis=1)})
result.to_csv('lgb_bal_5fold.csv', index=False)

#### Sample a small data set as validation set

In [70]:
sub_train = train_X[train_y==0].sample(n=18000, replace=False)
over_train = train_X[train_y==1].sample(n=1600, replace=False)

val_X = sub_train.append(over_train)
val_y = np.repeat([0, 1], [18000, 1600])

In [71]:
train_ind = list(set(train_X.index).difference(val_X.index))
train_X2 = train_X.iloc[train_ind]
train_y2 = train_y[train_ind]

In [73]:
train_data = lgb.Dataset(train_X2, label=train_y2, feature_name='auto', 
                                  categorical_feature=cat_feats)

test_data = lgb.Dataset(val_X, label=val_y, reference=train_data)

bst = lgb.train(param, train_data, 10000, valid_sets=[test_data], early_stopping_rounds=100)

[1]	valid_0's auc: 0.703557
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.710787
[3]	valid_0's auc: 0.713665
[4]	valid_0's auc: 0.713619
[5]	valid_0's auc: 0.718127
[6]	valid_0's auc: 0.718394
[7]	valid_0's auc: 0.71903
[8]	valid_0's auc: 0.719423
[9]	valid_0's auc: 0.719215
[10]	valid_0's auc: 0.72054
[11]	valid_0's auc: 0.720878
[12]	valid_0's auc: 0.721207
[13]	valid_0's auc: 0.725363
[14]	valid_0's auc: 0.725296
[15]	valid_0's auc: 0.725078
[16]	valid_0's auc: 0.724715
[17]	valid_0's auc: 0.724988
[18]	valid_0's auc: 0.724492
[19]	valid_0's auc: 0.725032
[20]	valid_0's auc: 0.725344
[21]	valid_0's auc: 0.725007
[22]	valid_0's auc: 0.725077
[23]	valid_0's auc: 0.725278
[24]	valid_0's auc: 0.725523
[25]	valid_0's auc: 0.725327
[26]	valid_0's auc: 0.725745
[27]	valid_0's auc: 0.725721
[28]	valid_0's auc: 0.726003
[29]	valid_0's auc: 0.727534
[30]	valid_0's auc: 0.727555
[31]	valid_0's auc: 0.728952
[32]	valid_0's auc: 0.728726
[33]	valid_0's auc: 

[277]	valid_0's auc: 0.769964
[278]	valid_0's auc: 0.770079
[279]	valid_0's auc: 0.770184
[280]	valid_0's auc: 0.770266
[281]	valid_0's auc: 0.770313
[282]	valid_0's auc: 0.770461
[283]	valid_0's auc: 0.770553
[284]	valid_0's auc: 0.770594
[285]	valid_0's auc: 0.770711
[286]	valid_0's auc: 0.7708
[287]	valid_0's auc: 0.770864
[288]	valid_0's auc: 0.770939
[289]	valid_0's auc: 0.770981
[290]	valid_0's auc: 0.771047
[291]	valid_0's auc: 0.771139
[292]	valid_0's auc: 0.771183
[293]	valid_0's auc: 0.771255
[294]	valid_0's auc: 0.771383
[295]	valid_0's auc: 0.771464
[296]	valid_0's auc: 0.771632
[297]	valid_0's auc: 0.771705
[298]	valid_0's auc: 0.771768
[299]	valid_0's auc: 0.771839
[300]	valid_0's auc: 0.771922
[301]	valid_0's auc: 0.771966
[302]	valid_0's auc: 0.772068
[303]	valid_0's auc: 0.77212
[304]	valid_0's auc: 0.772174
[305]	valid_0's auc: 0.772291
[306]	valid_0's auc: 0.772342
[307]	valid_0's auc: 0.772435
[308]	valid_0's auc: 0.772531
[309]	valid_0's auc: 0.772635
[310]	valid_0

[551]	valid_0's auc: 0.781546
[552]	valid_0's auc: 0.781567
[553]	valid_0's auc: 0.781562
[554]	valid_0's auc: 0.781544
[555]	valid_0's auc: 0.781623
[556]	valid_0's auc: 0.78165
[557]	valid_0's auc: 0.781712
[558]	valid_0's auc: 0.781691
[559]	valid_0's auc: 0.78173
[560]	valid_0's auc: 0.781733
[561]	valid_0's auc: 0.781741
[562]	valid_0's auc: 0.781691
[563]	valid_0's auc: 0.781663
[564]	valid_0's auc: 0.781674
[565]	valid_0's auc: 0.781721
[566]	valid_0's auc: 0.781788
[567]	valid_0's auc: 0.781822
[568]	valid_0's auc: 0.781844
[569]	valid_0's auc: 0.781848
[570]	valid_0's auc: 0.78186
[571]	valid_0's auc: 0.78189
[572]	valid_0's auc: 0.78189
[573]	valid_0's auc: 0.781881
[574]	valid_0's auc: 0.781907
[575]	valid_0's auc: 0.781873
[576]	valid_0's auc: 0.781857
[577]	valid_0's auc: 0.781876
[578]	valid_0's auc: 0.781886
[579]	valid_0's auc: 0.781863
[580]	valid_0's auc: 0.781875
[581]	valid_0's auc: 0.781885
[582]	valid_0's auc: 0.781932
[583]	valid_0's auc: 0.781961
[584]	valid_0's

[826]	valid_0's auc: 0.784008
[827]	valid_0's auc: 0.783979
[828]	valid_0's auc: 0.783966
[829]	valid_0's auc: 0.783973
[830]	valid_0's auc: 0.783944
[831]	valid_0's auc: 0.783898
[832]	valid_0's auc: 0.783876
[833]	valid_0's auc: 0.783908
[834]	valid_0's auc: 0.783886
[835]	valid_0's auc: 0.783892
[836]	valid_0's auc: 0.783908
[837]	valid_0's auc: 0.783938
[838]	valid_0's auc: 0.783941
[839]	valid_0's auc: 0.783941
[840]	valid_0's auc: 0.783927
[841]	valid_0's auc: 0.783923
[842]	valid_0's auc: 0.78392
[843]	valid_0's auc: 0.783911
[844]	valid_0's auc: 0.783879
[845]	valid_0's auc: 0.78386
[846]	valid_0's auc: 0.783882
[847]	valid_0's auc: 0.783888
[848]	valid_0's auc: 0.783888
[849]	valid_0's auc: 0.78389
[850]	valid_0's auc: 0.783911
[851]	valid_0's auc: 0.783964
[852]	valid_0's auc: 0.783965
[853]	valid_0's auc: 0.783987
[854]	valid_0's auc: 0.783997
[855]	valid_0's auc: 0.78402
[856]	valid_0's auc: 0.784037
[857]	valid_0's auc: 0.78404
[858]	valid_0's auc: 0.784063
[859]	valid_0's

[1098]	valid_0's auc: 0.784523
[1099]	valid_0's auc: 0.784535
[1100]	valid_0's auc: 0.784518
[1101]	valid_0's auc: 0.784488
[1102]	valid_0's auc: 0.784509
[1103]	valid_0's auc: 0.784538
[1104]	valid_0's auc: 0.784528
[1105]	valid_0's auc: 0.784542
[1106]	valid_0's auc: 0.784542
[1107]	valid_0's auc: 0.78458
[1108]	valid_0's auc: 0.784609
[1109]	valid_0's auc: 0.784582
[1110]	valid_0's auc: 0.784593
[1111]	valid_0's auc: 0.784611
[1112]	valid_0's auc: 0.784599
[1113]	valid_0's auc: 0.784609
[1114]	valid_0's auc: 0.784599
[1115]	valid_0's auc: 0.784624
[1116]	valid_0's auc: 0.784579
[1117]	valid_0's auc: 0.78459
[1118]	valid_0's auc: 0.784581
[1119]	valid_0's auc: 0.784584
[1120]	valid_0's auc: 0.784577
[1121]	valid_0's auc: 0.784579
[1122]	valid_0's auc: 0.784613
[1123]	valid_0's auc: 0.78465
[1124]	valid_0's auc: 0.784641
[1125]	valid_0's auc: 0.784643
[1126]	valid_0's auc: 0.784628
[1127]	valid_0's auc: 0.784629
[1128]	valid_0's auc: 0.78463
[1129]	valid_0's auc: 0.784614
[1130]	valid

In [74]:
result = pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': bst.predict(test)})
result.to_csv('lgb_bal.csv', index=False)